In [12]:
!pip install pymorphy2

In [13]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.stem import WordNetLemmatizer
import sklearn
import codecs
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, HashingVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier, RidgeClassifier, Perceptron
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.svm import LinearSVC
import pymorphy2
import seaborn as sns
sns.set_style("darkgrid")

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:

#FUNCTIONS------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------------
def list_to_str(arr):
    str_ = ''
    for rec in arr:
        str_+=rec
    return str_

def csv_to_list(arr):
    arr_list = []
    for row in arr:
        arr_list.append(list_to_str(row))
    return arr_list

def value_smiles_in_str(arr,str):
    count = 0
    for smile in arr:
        count+=str.count(smile)
    return count


def mistakes(arr,y,y_pred):
    for i in range(len(arr)):
        if y.iloc[i]!=y_pred[i]:
            print('True: ', y.iloc[i],' Prediction: ', y_pred[i],'\n')
            print(arr.iloc[i])
            
def df_preprocess_str(text):    
    reg = re.compile('[^а-яА-яa-zA-Z0-9 ]') #
    text = text.lower().replace("ё", "е")
    text = text.replace("ъ", "ь")
    text = text.replace("й", "и")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'сайт', text)
    text = re.sub('@[^\s]+', 'пользователь', text)
    text = reg.sub(' ', text)
    morph = pymorphy2.MorphAnalyzer()
    #lemmatizer = WordNetLemmatizer()
    text_ = []
    for word in text.split():
        text_.append(morph.parse(word)[0].normal_form)
    return text_

def get_vectorizer(str_): #'TF-IDF' sublinear_tf=True, max_df=0.5, stop_words='english'
  
    if str_ == 'HASH':
        vectorizer = HashingVectorizer(stop_words='english', alternate_sign=False
  ) #n_features=None
  
    if str_ == 'FP':#Feature Presence
        vectorizer = CountVectorizer(ngram_range=ngram_range,
                                         min_df=min_df,
                                         max_df=max_df)  # binary=True,stop_words='english'

    if str_ == 'TF-IDF':#Feature Presence    
        vectorizer = TfidfVectorizer(ngram_range=(1,2),sublinear_tf=True, max_df=0.5, stop_words='english') # ngram_range=(1,2),binary=True stop_words='english' ngram_range=ngram_range,min_df=min_df, max_df=max_df, sublinear_tf=True
    return vectorizer



def preprocess(df_train, df_test, str_):

# генерим матрицу TFIDF, методом главных компонент (PCA) редуцируем вектора до размерности 100
    all_df = pd.concat([df_train['text'],df_test['text']], axis = 0)
    vectorizer =  get_vectorizer(str_)
    vectorizer.fit(all_df)
    Mat_count_train = vectorizer.transform(df_train['text'])
    Mat_count_test = vectorizer.transform(df_test['text'])
    return Mat_count_train, Mat_count_test
  
def check_model(model,data_t, y_t, data_v, y_v):
    scores = cross_val_score(model, data_t, y_t, cv=3, scoring='accuracy')
    print(model,'\n Cross-validate: ', scores)
    model.fit(data_t,y_t)
    y_p = model.predict(data_v)
    print("Accuracy on valid data: ", accuracy_score(y_p,y_v))
    return y_p
  
def check_data_tf_idf(data_t, y_t, data_v, y_v):
    #forest = RandomForestClassifier(n_estimators=50)
    #check_model(forest,data_t, y_t, data_v, y_v)
    
    '''
    sgd_cl = SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=50,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)
    check_model(sgd_cl,data_t, y_t, data_v, y_v)
    
    sgd_cl_el = SGDClassifier(alpha=.0001, max_iter=50, penalty="elasticnet")
    check_model(sgd_cl_el,data_t, y_t, data_v, y_v)
    '''
    linSVC = LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l1', random_state=None, tol=0.001,
     verbose=0)
    check_model(linSVC,data_t, y_t, data_v, y_v)
    
    m_NB = MultinomialNB(alpha=.01)
    check_model(m_NB,data_t, y_t, data_v, y_v)
    
    b_NB = BernoulliNB(alpha=.005)
    check_model(b_NB,data_t, y_t, data_v, y_v)
    
    c_NB = ComplementNB(alpha=.001)
    check_model(c_NB,data_t, y_t, data_v, y_v)
    
    
def check_data_hash(data_t, y_t, data_v, y_v):
    #forest = RandomForestClassifier(n_estimators=50)
    #check_model(forest,data_t, y_t, data_v, y_v)
    
    '''
    sgd_cl = SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=50,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)
    check_model(sgd_cl,data_t, y_t, data_v, y_v)
    
    sgd_cl_el = SGDClassifier(alpha=.0001, max_iter=50, penalty="elasticnet")
    check_model(sgd_cl_el,data_t, y_t, data_v, y_v)
    
    linSVC = LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l1', random_state=None, tol=0.001,
     verbose=0)
    check_model(linSVC,data_t, y_t, data_v, y_v)
    '''
    ridge = RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='sag',
        tol=0.01)
    check_model(ridge,data_t, y_t, data_v, y_v)
    
    per = Perceptron(max_iter=70, tol=1e-3)
    check_model(per,data_t, y_t, data_v, y_v)    
#---------------------------------------------------------------------------------------------------------------

In [16]:
#READ DATA------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------------
positive_recalls = csv_to_list(csv.reader(codecs.open('/content/drive/My Drive/Colab Notebooks/NLP/positive_recalls.csv', 'rU', 'utf-8', errors='ignore')))
negative_recalls = csv_to_list(csv.reader(codecs.open('/content/drive/My Drive/Colab Notebooks/NLP/negative_recalls.csv', 'rU', 'utf-8', errors='ignore')))

columns =['id','tdate','tmane','ttype','trtw','trep','tfav','tstcount','tfol','tfrien','listcount']
positive_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP/positive.csv',names=columns, header=None,sep=';')
negative_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP/negative.csv',names=columns, header=None,sep=';')

DF_positive = pd.concat((positive_df, pd.DataFrame(positive_recalls,columns= ['text'], index = positive_df.index)), axis = 1)
DF_negative = pd.concat((negative_df, pd.DataFrame(negative_recalls,columns= ['text'], index = negative_df.index)), axis = 1)

DF_positive_ = DF_positive.sample(frac=1.0) # рандомизированный выбор по умолчанию
DF_negative_ = DF_negative.sample(frac=1.0)

df = pd.concat((DF_positive_, DF_negative_),axis = 0).sample(frac = 1) # объединяем и перемешиваем

y = df['ttype'].replace({-1:0, 1:1})
del(df['ttype'])
X_train, X_valid, y_train, y_valid = train_test_split(df, y, test_size=.15, random_state=42, stratify=y)
print(X_train.shape,y_train.shape, X_valid.shape, y_valid.shape)
#---------------------------------------------------------------------------------------------------------------
#DATA PREPROCESSING---------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------------

X_train['text'] = X_train['text'].apply(df_preprocess_str)
X_valid['text'] = X_valid['text'].apply(df_preprocess_str)

#---------------------------------------------------------------------------------------------------------------

(192808, 11) (192808,) (34026, 11) (34026,)


KeyboardInterrupt: ignored

In [4]:
#TF-IDF----------------------------------------------------------------------------
#----------------------------------------------------------------------------------
tf_idf_train, tf_idf_valid   = preprocess(X_train, X_valid,  str_ = 'TF-IDF')#'HASH'
print('OLD shapes: ', tf_idf_train.shape, tf_idf_valid.shape)
lsvc = LinearSVC(C=3.0, class_weight=None, dual=False, fit_intercept=True,intercept_scaling=1, loss='squared_hinge', max_iter=1000,multi_class='ovr', penalty='l1', random_state=None, tol=0.001,verbose=0).fit(tf_idf_train, y_train) #C=1.0, class_weight=None, dual=False, fit_intercept=True,intercept_scaling=1, loss='squared_hinge', max_iter=1000,multi_class='ovr', penalty='l1', random_state=None, tol=0.001,verbose=0
#C=0.25, penalty="l1", dual=False
model = SelectFromModel(lsvc, prefit=True, max_features  = None) #threshold = 1e-5,
tf_idf_train_new = model.transform(tf_idf_train)
tf_idf_valid_new = model.transform(tf_idf_valid)
print('\nNew shapes: ', tf_idf_train_new.shape, tf_idf_valid_new.shape)
check_data_tf_idf(tf_idf_train_new, y_train, tf_idf_valid_new, y_valid)

OLD shapes:  (192808, 1309149) (34026, 1309149)

New shapes:  (192808, 178576) (34026, 178576)
LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l1', random_state=None, tol=0.001,
          verbose=0) 
 Cross-validate:  [0.7596857  0.76391417 0.75923073]
Accuracy on valid data:  0.7529536236995239
MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True) 
 Cross-validate:  [0.81330325 0.8134715  0.8104685 ]
Accuracy on valid data:  0.7375242461646976
BernoulliNB(alpha=0.005, binarize=0.0, class_prior=None, fit_prior=True) 
 Cross-validate:  [0.81311654 0.81325367 0.80807232]
Accuracy on valid data:  0.7403162287662376
ComplementNB(alpha=0.001, class_prior=None, fit_prior=True, norm=False) 
 Cross-validate:  [0.81219854 0.8112776  0.80883474]
Accuracy on valid data:  0.735760888732146


In [5]:
#PREDICT on tf-idf data
y_1 = check_model(LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l1', random_state=None, tol=0.001,
     verbose=0),tf_idf_train_new, y_train, tf_idf_valid_new, y_valid)
y_2 = check_model(BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True),tf_idf_train_new, y_train, tf_idf_valid_new, y_valid)

LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l1', random_state=None, tol=0.001,
          verbose=0) 
 Cross-validate:  [0.75973238 0.76386749 0.75915294]
Accuracy on valid data:  0.7529536236995239
BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True) 
 Cross-validate:  [0.79094445 0.79154802 0.78498187]
Accuracy on valid data:  0.7495738552871334


In [6]:
#HASH----------------------------------------------------------------------------
#----------------------------------------------------------------------------------
tf_idf_train, tf_idf_valid   = preprocess(X_train, X_valid,  str_ = 'HASH')#'HASH'
print('OLD shapes: ', tf_idf_train.shape, tf_idf_valid.shape)
lsvc = LinearSVC(C=2.0, class_weight=None, dual=False, fit_intercept=True,intercept_scaling=1, loss='squared_hinge', max_iter=1000,multi_class='ovr', penalty='l1', random_state=None, tol=0.001,verbose=0).fit(tf_idf_train, y_train) #C=1.0, class_weight=None, dual=False, fit_intercept=True,intercept_scaling=1, loss='squared_hinge', max_iter=1000,multi_class='ovr', penalty='l1', random_state=None, tol=0.001,verbose=0
#C=0.25, penalty="l1", dual=False
model = SelectFromModel(lsvc, prefit=True, max_features  = None) #threshold = 1e-5,
tf_idf_train_new = model.transform(tf_idf_train)
tf_idf_valid_new = model.transform(tf_idf_valid)
print('\nNew shapes: ', tf_idf_train_new.shape, tf_idf_valid_new.shape)
check_data_hash(tf_idf_train_new, y_train, tf_idf_valid_new, y_valid)

OLD shapes:  (192808, 1048576) (34026, 1048576)

New shapes:  (192808, 59728) (34026, 59728)
RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None, solver='sag',
                tol=0.01) 
 Cross-validate:  [0.76016804 0.76049106 0.7573947 ]
Accuracy on valid data:  0.7471051548815612
Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=70, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=0, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False) 
 Cross-validate:  [0.73880504 0.72185657 0.74012354]
Accuracy on valid data:  0.7102803738317757


In [7]:
#PREDICT on HASH data
y_3 = check_model(RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='sag',
        tol=0.01),tf_idf_train_new, y_train, tf_idf_valid_new, y_valid)

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None, solver='sag',
                tol=0.01) 
 Cross-validate:  [0.7601836  0.76075557 0.75750362]
Accuracy on valid data:  0.7471933227531887


In [0]:
y_total = [y_1[i]*y_2[i] ^ y_1[i]*y_3[i] ^ y_2[i]*y_3[i] for i in range(len(y_1))]

In [9]:
print('Accuracy: ', accuracy_score(y_valid,y_total))
print('Precision: ', precision_score(y_valid,y_total))
print('Recall: ', recall_score(y_valid,y_total))
print('F1: ', f1_score(y_valid,y_total))
print('Confusion matrix:\n', confusion_matrix(y_valid, y_total))

Accuracy:  0.7583612531593488
Precision:  0.7477056657690828
Recall:  0.7893484945176075
F1:  0.7679629734153637
Confusion matrix:
 [[12198  4591]
 [ 3631 13606]]
